In [2]:
urls = [
    "https://secure.glaciernationalparklodges.com/booking/lodging-flex-search?destination=ALL&adults=2&children=0&infants=0&animals=0&rateCode=&rateType=&nights=1&dateFrom=07-01-2024",
    "https://secure.glaciernationalparklodges.com/booking/lodging-flex-search?destination=ALL&adults=2&children=0&infants=0&animals=0&rateCode=&rateType=&nights=2&dateFrom=07-01-2024",
    "https://secure.glaciernationalparklodges.com/booking/lodging-flex-search?destination=ALL&adults=2&children=0&infants=0&animals=0&rateCode=&rateType=&nights=1&dateFrom=08-01-2024",
    "https://secure.glaciernationalparklodges.com/booking/lodging-flex-search?destination=ALL&adults=2&children=0&infants=0&animals=0&rateCode=&rateType=&nights=2&dateFrom=08-01-2024"
]

In [39]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

options = Options()
options.headless = True 
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

url = "https://secure.glaciernationalparklodges.com/booking/lodging-flex-search?destination=ALL&adults=2&children=0&infants=0&animals=0&rateCode=&rateType=&nights=1&dateFrom=07-01-2024"

driver.get(url)

wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'price-calendar-container')))

driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(3) 

soup = BeautifulSoup(driver.page_source, 'html.parser')

driver.quit()

price_calendar = soup.find('div', class_='price-calendar')

date_rows = price_calendar.find_all('div', class_='day-row-wrapper')


dates_prices = []

for date_cell in date_rows:
    date = date_cell.find('div', class_='day-row').find('span', class_='day-title').text.strip()
    hotel_prices = []
    price_cells = date_cell.find_all('div', class_='price-cell')


    # # this will check all hotels

    # for price_cell in price_cells:
    #     if price_cell.find('span', class_='sold-out'):
    #         price = "Unavailable"
    #     else:
    #         price = price_cell.get_text(strip=True)
    #     hotel_prices.append(price)

    price_cell = price_cells[4]  # this will only check the 5th hotel

    if price_cell.find('span', class_='sold-out'):
            price = "Unavailable"
    else:
        price = price_cell.get_text(strip=True)
    hotel_prices.append(price)
    
    if any(price != "Unavailable" for price in hotel_prices): 
        dates_prices.append((date, hotel_prices))

print("Availabilities: ")
for date, hotels_prices in dates_prices:
    available_prices = [(idx + 1, price) for idx, price in enumerate(hotels_prices) if price != "Unavailable"]
    print(f"  {date}")
    for hotel, price in available_prices:
        print(f"    Price: {price}")

Availabilities: 
  Jul 1
    Price: $271
  Jul 31
    Price: $239


In [48]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

def check_availability(url):
    options = Options()
    options.headless = True 
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    driver.get(url)

    wait = WebDriverWait(driver, 18)
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'price-calendar-container')))

    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # time.sleep(3) 

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    driver.quit()

    price_calendar = soup.find('div', class_='price-calendar')

    date_rows = price_calendar.find_all('div', class_='day-row-wrapper')

    dates_prices = []

    for date_cell in date_rows:
        date = date_cell.find('div', class_='day-row').find('span', class_='day-title').text.strip()
        price_cells = date_cell.find_all('div', class_='price-cell')
        if len(price_cells) >= 5:
            price_cell = price_cells[4]  # Check only the 5th hotel
            if price_cell.find('span', class_='sold-out'):
                price = "Unavailable"
            else:
                price = price_cell.get_text(strip=True)
            
            if price != "Unavailable":
                dates_prices.append((date, price))
    
    return dates_prices

urls = [
    "https://secure.glaciernationalparklodges.com/booking/lodging-flex-search?destination=ALL&adults=2&children=0&infants=0&animals=0&rateCode=&rateType=&nights=1&dateFrom=07-01-2024",
    "https://secure.glaciernationalparklodges.com/booking/lodging-flex-search?destination=ALL&adults=2&children=0&infants=0&animals=0&rateCode=&rateType=&nights=2&dateFrom=07-01-2024",
    "https://secure.glaciernationalparklodges.com/booking/lodging-flex-search?destination=ALL&adults=2&children=0&infants=0&animals=0&rateCode=&rateType=&nights=1&dateFrom=08-01-2024",
    "https://secure.glaciernationalparklodges.com/booking/lodging-flex-search?destination=ALL&adults=2&children=0&infants=0&animals=0&rateCode=&rateType=&nights=2&dateFrom=08-01-2024"
]

results = {}

for url in urls:
    nights = url.split("nights=")[1].split("&")[0]
    date_from = url.split("dateFrom=")[1]

    key = f"  {nights} night(s) stay"
    if key not in results:
        results[key] = []

    results[key].extend(check_availability(url))

print("Swiftcurrent Availabilities:")
for key in sorted(results.keys()):
    print(f"{key}:")
    if len(results[key]) == 0:
        print("    None found")
    else:
        for date, price in results[key]:
            print(f"    Date: {date}")
            print(f"      Price: {price}")


Swiftcurrent Availabilities:
  1 night(s):
    Date: Aug 4
      Price: $185
  2 night(s):
    None found
